# TechXchange Japan 2024: さわってみようベクトル・データベース watsonx.dataでRAG体験


生成AIの回答精度を向上させるために、自社内のデータを活用してみましょう！<br>
ベクトル・データベース + 大規模言語モデル（LLM）で構成されるRAGのアーキテクチャーを使えば、自社内の情報で生成AIチャットボットが作成できます。<br>
当ハンズオンでは「TechXchange Japan 2024」に関するデータを使ってRAGを構成し、「TechXchange Japan 2024」に関することを教えてくれるチャットボットを作成します。<br>
(以下ベクトル・データベースはベクトルDBと表記します。)<br>


具体的には大規模言語モデル（LLM）を使用したアプリケーション開発のためのオープンソース・オーケストレーション・フレームワーク[LangChain](https://python.langchain.com/docs/introduction/)を使って、wastosonx.dataのベクトルDB **Milvus**に「TechXchange Japan 2024」に関するデータをロードし、watson.aiで提供されているLLMを使用してRAGを構成し、「TechXchange Japan 2024」のことを回答してくれるChatbotを作ってみます。

ハンズオンは以下の順序で実行します:

1. Excelをベクトル化してベクトルDB Milvusに入れよう！
2. **ベクトルDB Milvusに入ったデータで類似検索してみよう!** [**当notebook**]
3. ベクトルDB Milvusとwatsonx.ai LLMでRAGを構成して、質問をしてみよう!
4. ベクトルDB Milvusとwatsonx.ai LLMでRAGを構成して、チャットアプリを作成してみよう!

このハンズオンのガイドは[**こちら**](https://ibm.biz/20241127-rag-handson)にあります。


## 2. ベクトルDB Milvusに入ったデータで類似検索してみよう! 

**実行は　セルを選択(クリック)して**

- **Windows： Ctrl + Enter** 
- **Mac: ⌘ (command) + Enter 　または Ctrl + Enter**

**実行して次のセルを選択は　セルを選択(クリック)して**

- **Windows： Shift + Enter** 
- **Mac: Shift + Enter**

セルの左側に[*]が表示されている場合は実行中です。<br>
[1]のように数字が入っている場合は、実行が完了しています。

### 1. 必要なライブラリーのインストール

In [ ]:
print("pip install start")
!pip install -Uq 'ibm-watsonx-ai>=1.1.15'
!pip install -Uq 'langchain>=0.3.3'
!pip install -Uq 'langchain-ibm>=0.3.1'
!pip install -Uq 'langchain-milvus>=0.1.6'
!pip install -Uq 'langchain-community>=0.3.2'
!pip install -Uq 'pymilvus>=2.4.8'
print("pip install completed")

**インストール終了後、一旦カーネルを再起動してください** <br>

**手順:**
- 上部のメニュー「Karnel」から「Restart Karnel and Clear Outputs of All Cells...」をクリック
- 「Restart Kernel?」 のダイアログが表示されるので、「Restart」をクリック

### 2. apikeyの設定 

- 実行するとテキスト入力ボックスが表示されるので、事前に取得したapikeyをに入れてEnter Keyを押してください

- ハンズオン環境で実行の場合は、`IBM Cloud Service API key`を入力してください。　よくわからない場合は[こちら](https://github.com/IBM/japan-technology/tree/main/techxchange/2024-watsonx-handson-1/01_techzone_use_environments.md)の2-3を参照してください。

In [ ]:
import getpass
apikey = getpass.getpass("apikeyを入れてEnter Keyを押してください:")

### 3. Milvus接続情報の設定

実行すると順に以下の2つのテキスト入力ボックスが表示されるので、事前に取得したMilvus接続情報の値を入力してください。
- milvus GRPC ホストを入れてEnter Keyを押してください →  GRPC ホストの値を入力  

- milvus GRPC ポートを入れてEnter Keyを押してください　→ GRPC ポートの値をを入力


Milvus接続情報の詳細取得手順は[こちら](https://github.com/IBM/japan-technology/tree/main/techxchange/2024-watsonx-handson-1/watsonx_data_get_milvus_info.md) を参照

簡略な接続手順は以下です:
- watsonx.dataの画面を開く
- ナビゲーションメニューから「インフラストラクチャー・マネージャー」を選択
- サービス「Milvus」をクリック
- タイプの下の「接続の詳細を見る」をクリック
- GRPC ホストの値とGRPC ポートの値を取得
  
collection名は`techxchange_line_data`としています。

In [ ]:
milvus_host=input("milvus GRPC ホストを入れてEnter Keyを押してください: ")
milvus_port=input("milvus GRPC ポートを入れてEnter Keyを押してください: ")

In [ ]:
# Milvus接続情報パラメータののセット
my_connection_args ={
 'uri': f'https://{milvus_host}:{milvus_port}', 
 'token': f'ibmlhapikey:{apikey}'
}

my_collection =  'techxchange_line_data'

### 4. watsonx.ai Project idの設定

Watson Studioで実行する場合は、このノートブックが実行されるプロジェクトからProject idを取得します。
Watson Studio以外で実行する場合は、Project idを入力してください。

**Hint**: `project_id` はプロジェクトを表示し、管理タブから `project_id` を取得可能です.

In [ ]:
import os
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Project idを入力してください (入力後enter): ")

### 5.watsonx.aiのAuthentication用のエンドポイントのURLの設定

Waston Machine Learningのインスタンスを作成したリージョンで決まります。
https://ibm.github.io/watson-machine-learning-sdk/setup_cloud.html#authentication　より

- Dallas: https://us-south.ml.cloud.ibm.com
- London: https://eu-gb.ml.cloud.ibm.com
- Frankfurt: https://eu-de.ml.cloud.ibm.com
- Tokyo: https://jp-tok.ml.cloud.ibm.com

今回はダラスのWaston Machine Learningのインスタンスを使っているので`https://us-south.ml.cloud.ibm.com`を使います。

In [ ]:
watsonx_url = "https://us-south.ml.cloud.ibm.com" #watsonx.aiのAuthentication用のエンドポイントのURL

### 6. 必要ライブラリーのImport

In [ ]:
import pandas as pd
from langchain.schema.document import Document
import json
from langchain_milvus import Milvus
import os
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes

### 7. Embeddingモデルの取得
ベクトル化した時と同じモデル`intfloat/multilingual-e5-large`を使います<br>
- https://huggingface.co/intfloat/multilingual-e5-large
- https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models-embed.html?context=wx&locale=ja#multilingual-e5-large

LangChainで使用できるwatsonx.aiのEmbeddingモデル`intfloat/multilingual-e5-large`があるので、今回はこちらを使用します:
- https://python.langchain.com/docs/integrations/text_embedding/ibm_watsonx/
  
---

>尚、`intfloat/multilingual-e5-large`はオープンソースで公開されているので、watsonx.aiのEmbeddingモデルを使用しなくとも、ローカルにダウンロードすることで使用可能です。
その場合のコードはこちらです(今回は使用しません)

>```python
>from langchain_huggingface import HuggingFaceEmbeddings
>from tqdm.autonotebook import tqdm
>embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

>```
> 上記コード実行時の`TqdmExperimentalWarning`のWarningは無視でよいです

In [ ]:
# watsonx.aiのEmbeddingモデル取得
embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 512,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

embeddings = WatsonxEmbeddings(
    model_id="intfloat/multilingual-e5-large",
    url=watsonx_url ,
    apikey=apikey,
    project_id=project_id
    )

### 8.  ベクトルDB Milvusに接続

In [ ]:
from langchain_milvus import Milvus

vector_db = Milvus(
    embeddings,
    connection_args =my_connection_args,
    collection_name = my_collection
)

### 9.  挿入データの確認

Milvus DBにロードした内容をDataFrameにダンプして表示させます。

In [ ]:
from pymilvus import MilvusClient
import pandas as pd

pk_list=vector_db.get_pks(expr="pk > 0")

client = MilvusClient(uri=my_connection_args['uri'], token=my_connection_args['token'])

res = client.get(
    collection_name= my_collection,
    ids=pk_list
)

for i, milvus_rec in enumerate(res):
    vecter_data = milvus_rec['vector']
    res[i]['vector']= "["+", ".join(map(str, vecter_data))+"]"
    res[i]['pk']= str(milvus_rec['pk'])


df_s = pd.DataFrame.from_dict(res).reindex(columns=['ID', 'Category', 'text', 'pk', 'vector'])
df_s

### 10. テキスト類似検索してみます
基本、類似度が高い順でリストされます。いろいろなオプションで検索してみます。

#### 10-1. オプションなし デフォルト

In [ ]:
# オプションなし
query = "IBM TechXchange Japanとは?"
docs = vector_db.similarity_search(query)

for doc in docs:
    print({"content": doc.page_content[0:100], "metadata": doc.metadata} )
    print("---------")

#### 10-2. 結果の取得数をkで指定(デフォルトは4)

In [ ]:
# 結果の取得数をkで指定(デフォルトは4)

docs = vector_db.similarity_search(query, k=10)

for doc in docs:
    print({"content": doc.page_content[0:100], "metadata": doc.metadata} )
    print("---------")

#### 10-3. 類似度のスコアも一緒に出力
- 類似度のスコアも一緒に出してみます
- スコアはコサイン類似度の場合は、1に近いほど類似度が高いです。

In [ ]:
# 類似度のスコアも一緒に出してみます
# スコアはコサイン類似度の場合は、1に近いほど類似度が高いです。

docs = vector_db.similarity_search_with_score(query, k=10)
for doc, score in docs:
    print({"score": score, "content": doc.page_content[0:100], "metadata": doc.metadata} )
    print("---------")

#### 10-4. 'Category': '概要'　でフィルター

In [ ]:
# 'Category': '概要'　でフィルターしてみます
docs = vector_db.similarity_search_with_score(query, k=10, expr="Category=='概要'")
for doc, score in docs:
    print({"score": score, "content": doc.page_content[0:100], "metadata": doc.metadata} )
    print("---------")

### これで「2. ベクトルDB Milvusに入ったデータで類似検索してみよう!」は完了です。<br>


#### Notebookを保存する場合は、右上の保存アイコンをクリックして保存してください。

- <img width="400" alt="" src="https://github.com/IBM/japan-technology/blob/main/techxchange/2024-watsonx-handson-1/images/save_notebook.jpg?raw=true">
<br>
<br>

#### プロジェクトの画面に戻る場合は、右上のプロジェクト名をクリックしてください。

- <img width="400" alt="" src="https://github.com/IBM/japan-technology/blob/main/techxchange/2024-watsonx-handson-1/images/return_to_project.jpg?raw=true">
<br>
<br>

#### Notebookを開いたままでプロジェクトの画面を表示するには、上部のプロジェクト名を右クリックし、「新しいタブで開く」でプロジェクトを新しいタブで開いてください。

- <img width="500" alt="" src="https://github.com/IBM/japan-technology/blob/main/techxchange/2024-watsonx-handson-1/images/open_project_from_notebook.jpg?raw=true">
<br>
<br>

#### 次の「3. ベクトルDB Milvusとwatsonx.ai LLMでRAGを構成して、質問をしてみよう!」に進んでください。